In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import visdom
import h5py

C:\Users\Utkarsh Sah\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
from sklearn.decomposition import PCA
from scipy.spatial.transform import Rotation as R

In [4]:
from preprocessing_utils import process_sat_data

In [5]:
data_train = pd.read_csv('../data/train.csv')
data_test = pd.read_csv('../data/track1/test.csv')

In [6]:
train_data = data_train.copy(deep=True)
test_data = data_test.copy(deep=True)

In [7]:
process_sat_data(train_data, test_data, scale=10000)

# Arrange Data: 24-Point Ellipse Format

In [80]:
def get_sat_ellipse_ids(n):
    ids = []
    incomplete = (n%24)
    for i in range(n-incomplete):
        ids.append(int(i/24))
    prev = ids[-1]
    for i in range(incomplete):
        ids.append(prev)
    return ids

In [81]:
def generate_ellipse_ids(data):
    ids = []
    counts = np.unique(data['sat_id'], return_counts=True)[1]
    for count in counts:
        sat_ellipse_ids = get_sat_ellipse_ids(count)
        ids += sat_ellipse_ids
    return ids

In [82]:
def arrange_data(train_data, test_data):
    cols_sim = ['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim', 'sat_id', 'id']
    cols_real = ['x', 'y', 'z', 'Vx', 'Vy', 'Vz', 'sat_id']
    data_sim = train_data.loc[:, cols_sim].append(test_data.loc[:, cols_sim])
    data_sim = data_sim.sort_values('id').drop('id', axis=1)
    data_real = train_data.loc[:, cols_real]
    data_sim['ellipse_id'] = generate_ellipse_ids(data_sim)
    data_real['ellipse_id'] = generate_ellipse_ids(data_real)
    return [data_sim, data_real]

In [83]:
data_sim, data_real = arrange_data(train_data, test_data)

In [84]:
def format_data(pos, vel):
    last_index = int(len(pos)/24) * 24
    pos = pos[:last_index, :].reshape(-1,24,3)
    vel = vel[:last_index, :].reshape(-1,24,3)
    return [pos, vel]

In [85]:
pos_sim = {}
vel_sim = {}
pos_real = {}
vel_real = {}

In [86]:
for sat_id in range(600):
    sat_sim = data_sim[data_sim['sat_id'] == sat_id].loc[:, ['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']].values
    sat_real = data_real[data_real['sat_id'] == sat_id].loc[:, ['x', 'y', 'z', 'Vx', 'Vy', 'Vz']].values
    pos_sim[sat_id], vel_sim[sat_id] = format_data(sat_sim[:, [0,1,2]], sat_sim[:, [3,4,5]])
    pos_real[sat_id], vel_real[sat_id] = format_data(sat_real[:, [0,1,2]], sat_real[:, [3,4,5]])

# Compute Transformation

3D_Point . Inv(M) = 2D_Point <br>
2D_Point . M = 3D_Point

In [87]:
class Ellipse(nn.Module):
    
    def __init__(self, name, a, b, cx, cy):
        super(Ellipse, self).__init__()
        self.name = name
        self.alpha = nn.Parameter(1/(a**2))
        self.beta = nn.Parameter(1/(b**2))
        self.cx = nn.Parameter(cx)
        self.cy = nn.Parameter(cy)
        self.loss = -1
    
    def forward(self, data):
        x = data[:, 0] - self.cx
        y = data[:, 1] - self.cy
        res = (self.alpha * x**2 + self.beta * y**2 - 1) ** 2
        return res
    
    def get_parameters(self):
        a = np.sqrt(1/self.alpha.item())
        b = np.sqrt(1/self.beta.item())
        cx = self.cx.item()
        cy = self.cy.item()
        loss = self.loss
        return [a, b, loss]

In [88]:
def fit_ellipse(transformed, iterations=2500, learning_rate=0.001):
    transformed = torch.from_numpy(transformed).float()
    a = 0.5 * (transformed[:, 0].max() - transformed[:, 0].min())
    b = 0.5 * (transformed[:, 1].max() - transformed[:, 1].min())
    cx = torch.mean(transformed[:, 0])
    cy = torch.mean(transformed[:, 1])
    name = 's' + str(sat_id) + 'e' + str(ellipse_id)
    ellipse = Ellipse(name, a, b, cx, cy)
    optim = torch.optim.Adam(ellipse.parameters(), learning_rate)
    for itr in range(iterations):
        optim.zero_grad()
        pred = ellipse(transformed)
        loss = torch.sum(pred)
        loss.backward()
        optim.step()
    ellipse.loss = loss.item()
    return ellipse

In [89]:
def get_pca_estimates(ellipse_data):
    pca = PCA(3)
    pca.fit(ellipse_data)
    transformation = np.append(pca.components_, [ellipse_data.mean(axis=0)], axis=0)
    transformation = np.concatenate([transformation, [[0],[0],[0],[1]]], axis=1)
    return transformation

In [90]:
def compute_transformation(sat_id, ellipse_id, ellipse_data):
    transformation = get_pca_estimates(ellipse_data)
    ellipse_data = np.concatenate([ellipse_data, np.ones((24,1))], axis=1)
    transformed_data = ellipse_data.dot(np.linalg.inv(transformation))
    ellipse = fit_ellipse(transformed_data, 2500, 0.001)
    center_2d = np.array([ellipse.cx, ellipse.cy, 0, 1])
    center_3d = center_2d.dot(transformation)
    transformation[-1] = center_3d
    return [transformation, ellipse]

# Data Generation

Format: <br>
sat_id, ellipse_id, major, minor, loss, rotation (3x3), translation (1x3)

In [94]:
trans_sim = []
trans_real = []

In [95]:
for sat_id, sat_data in pos_sim.items():
    for ellipse_id, ellipse_data in enumerate(sat_data):
        transformation, ellipse = compute_transformation(sat_id, ellipse_id, ellipse_data)
        R = transformation[:3, :3].ravel()
        T = transformation[3, :3]
        ids = [sat_id, ellipse_id]
        ellipse_parameters = ellipse.get_parameters()
        row = np.concatenate([ids, ellipse_parameters, R, T], axis=0)
        trans_sim.append(row)
        print('Satellite ID: {}, Ellipse Id: {} | Loss: {} [Sim]'.format(sat_id, ellipse_id, ellipse_parameters[-1]))
        break
    df_sim = pd.DataFrame(trans_sim)
    df_sim.to_csv('../data/data_sim.csv', index=False)
    print ('Satellite ID', sat_id, 'Saved')

Satellite ID: 0, Ellipse Id: 0 | Loss: 1.2777285007814498e-08
Satellite ID 0 Saved
Satellite ID: 1, Ellipse Id: 0 | Loss: 5.7523866558994996e-08
Satellite ID 1 Saved
Satellite ID: 2, Ellipse Id: 0 | Loss: 0.0006194846937432885
Satellite ID 2 Saved
Satellite ID: 3, Ellipse Id: 0 | Loss: 0.06424389034509659
Satellite ID 3 Saved


KeyboardInterrupt: 

In [96]:
for sat_id, sat_data in pos_real.items():
    for ellipse_id, ellipse_data in enumerate(sat_data):
        transformation, ellipse = compute_transformation(sat_id, ellipse_id, ellipse_data)
        R = transformation[:3, :3].ravel()
        T = transformation[3, :3]
        ids = [sat_id, ellipse_id]
        ellipse_parameters = ellipse.get_parameters()
        row = np.concatenate([ids, ellipse_parameters, R, T], axis=0)
        trans_real.append(row)
        print('Satellite ID: {}, Ellipse Id: {} | Loss: {} [Real]'.format(sat_id, ellipse_id, ellipse_parameters[-1]))
        break
    df_real = pd.DataFrame(trans_real)
    df_real.to_csv('../data/data_real.csv', index=False)
    print ('Satellite ID', sat_id, 'Saved')

Satellite ID: 0, Ellipse Id: 0 | Loss: 3.128792513962253e-07 [Real]
Satellite ID 0 Saved
Satellite ID: 1, Ellipse Id: 0 | Loss: 7.44943235986284e-06 [Real]
Satellite ID 1 Saved
Satellite ID: 2, Ellipse Id: 0 | Loss: 0.000582799082621932 [Real]
Satellite ID 2 Saved
Satellite ID: 3, Ellipse Id: 0 | Loss: 0.06324034929275513 [Real]
Satellite ID 3 Saved
Satellite ID: 4, Ellipse Id: 0 | Loss: 3.391565428501053e-07 [Real]
Satellite ID 4 Saved
Satellite ID: 5, Ellipse Id: 0 | Loss: 5.061700676378678e-07 [Real]
Satellite ID 5 Saved
Satellite ID: 6, Ellipse Id: 0 | Loss: 0.0014275091234594584 [Real]
Satellite ID 6 Saved
Satellite ID: 7, Ellipse Id: 0 | Loss: 0.3032127320766449 [Real]
Satellite ID 7 Saved
Satellite ID: 8, Ellipse Id: 0 | Loss: 1.2872040144884522e-07 [Real]
Satellite ID 8 Saved
Satellite ID: 9, Ellipse Id: 0 | Loss: 0.007866712287068367 [Real]
Satellite ID 9 Saved
Satellite ID: 10, Ellipse Id: 0 | Loss: 6.638579748141638e-07 [Real]
Satellite ID 10 Saved
Satellite ID: 11, Ellipse 

KeyboardInterrupt: 